**What is convolution neural network**

Convolutional neural networks, also known as convnet, or CNNs, are a
special kind of neural network for processing data that has a known
grid-like topology like time series data(1D) or images(2D).

**CNN vs ANN** 

1. Overfitting (as there are more learnable parameters in ann compared to cnn , make it more complex model).

2. Loss of imp info like spatial arrangement of pixels.

**Convolution layer and Filters**

1. Edge Detection (Convolution Operation) : They do feature extraction like first they extract edges , then shapes etc. 

2. Each filter is a matrix which has kernel size (eg : 3 => (3,3)) , which is convolve over an area of image (receptive field) , then points at same position first are multiplied then added for a whole.

3. For single channel image : (n , n) conv (f,f)  => (n - f + 1 , n - f + 1)

4. For multi-channel image : (n , n , c) conv (f,f,c)  => (n - f + 1 , n - f + 1)


**Padding**

1. We can notice when convolution is done , we get smaller size .After many layers → The image shrinks too much , information gets lost.

2. Without padding, pixels near the edges and edge features get used less for convolution.

3. Padding adds a border around the image to resolve this issue.

4. There are two types : valid (no padding , image shrinks) and same (padding applied , output size same as input size).

5. Output image size : (n + 2p - f + 1, n - 2p + f + 1)

**Strides**

1. Strides are required when high level feature is required.

2. It decrease also the computing.

3. Output image size : [ (n + 2p - f)/s + 1, (n + 2p - f)/s + 1 ]  (when padding applied) , it considers floor value in division.

**Problem with Convolution**

[32 images](228 , 228 , 3) conv (3 , 3 , 3) [100 filters]  => ( 226 , 226, 100) * 32 which is around 634 MB if each value around 4 bytes. (total value * 4 / 1024 ^ 2)

1. **Memory issue** :  Above example shows how much memory does convolution takes place.

2. **Translation Variance** : Features become location dependant.


**Pooling** 

1. Pooling down sample you image. (using max pooling , min pooling etc.)

2. It helps to reduce the size and Translation invariance (check slides).

3. In case of max_pooling it leads in enhanced features.

4. There is no need of training. (as there is no training parameters)

5. But in some cases where object location is considered , avoid to use pooling.

**CNN Architecture**

| Layer               | Parameters                 | Input Shape  | Output Shape | Notes                                 |
| ------------------- | -------------------------- | ------------ | ------------ | ------------------------------------- |
| **Input**           | —                          | (3, 28, 28)  | (3, 28, 28)  | Image with 3 RGB channels                        |
| **Conv Layer 1**    | 32 filters, 3×3, padding=1 | (3, 28, 28)  | (32, 28, 28) | Filters automatically span 3 channels |
| **MaxPool 1**       | 2×2, stride=2              | (32, 28, 28) | (32, 14, 14) | Reduces spatial size                  |
| **Conv Layer 2**    | 64 filters, 3×3, padding=1 | (32, 14, 14) | (64, 14, 14) | Filters span 32 channels              |
| **MaxPool 2**       | 2×2, stride=2              | (64, 14, 14) | (64, 7, 7)   | Still 3D tensor                       |
| **Flatten**         | —                          | (64, 7, 7)   | (3136,)      | 64×7×7 = 3136                         |
| **Hidden FC Layer** | 64 neurons                 | (3136,)      | (64,)        | Fully connected                       |
| **Output Layer**    | 10 neurons                 | (64,)        | (10,)        | Class scores                          |


**BackPropagation in CNN**

1. The architecture can be seen as two parts.

2. First part include operations like convolution , pooling and flattening.

3. Second part can be seen as oprations like matrix muliplication in ANN .

4. Check the slides to see how gradients is calculated.

**CODE**